# Binary Document Classification with Feed-forward Neural Network model

**This project aims to classify texts from the NARRATIVE field into two classes ( Major or Minor).**

It is a classification algorithm which consists of a number of neurons organized in layers. Every unit in a layer is connected with all the units in the previous layer. These connections are not all equal: each connection may have a different strength or weight.
Data enters at the inputs and passes through the network, layer by layer, until it arrives at the outputs. There is no feedback between layers and hence they are called feedforward neural networks.
In this task, using the narrative column of mining dataset to determine the level(Minor or Major) of an injury.

The pipeline below consists of classes that converts the text input(in our case the NARRATIVE column) to vectorized minibatches.

#### Importing Libraries

In [ ]:
import collections
from collections import Counter
import string
import re
import nltk
nltk.download('wordnet')
import os
import json
from tqdm import tqdm 

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from torch.utils.data import Dataset
class InjuryDataset(Dataset):
    def __init__(self, injury_df, vectorizer):
        """
        Args:
        injury_df (pandas.DataFrame): the dataset
        vectorizer (InjuryVectorizer): vectorizer instantiated from dataset
        """
        self.injury_df = injury_df
        self._vectorizer = vectorizer
        self.train_df = self.injury_df[self.injury_df.split=='train']
        self.train_size = len(self.train_df)
        self.val_df = self.injury_df[self.injury_df.split=='val']
        self.validation_size = len(self.val_df)
        self.test_df = self.injury_df[self.injury_df.split=='test']
        self.test_size = len(self.test_df)
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.validation_size),
                            'test': (self.test_df, self.test_size)}
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, binary_injury_data_csv):
        """Load dataset and make a new vectorizer from scratch
        Args:
            binary_injury_data_csv (str): location of the dataset
        Returns:
            an instance of InjuryDataset
        """
        injury_df = pd.read_csv(binary_injury_data_csv)
        return cls(injury_df, InjuryVectorizer.from_dataframe(injury_df))
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer
    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    def __len__(self):
        return self._target_size
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        injury_vector = \
        self._vectorizer.vectorize(row.NARRATIVE)
        injury_level_index = \
        self._vectorizer.injury_level_vocab.lookup_token(row.injury_level)
        return {'x_data': injury_vector,
                'y_target': injury_level_index}
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
        batch_size (int)
        Returns:
        number of batches in the dataset
        """
        return len(self) // batch_size

In [ ]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

### Vectorizer

The text dataset is converted to a vectorized minibatch by iterating through the tokens of an input data point and converting each token to its integer form. 

In [ ]:
class InjuryVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, narrative_vocab, injury_level_vocab):
        """
        Args:
            narrative_vocab (Vocabulary): maps words to integers
            injury_level_vocab (Vocabulary): maps class labels to integers
        """
        self.narrative_vocab = narrative_vocab
        self.injury_level_vocab = injury_level_vocab
    def vectorize(self, narrative):
        """Create a collapsed one hot vector for the narrative
        Args:
            narrative (str): the narrative
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.narrative_vocab), dtype=np.float32)
        for token in narrative.split(" "):
            if token not in string.punctuation:
                one_hot[self.narrative_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, injury_df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            injury_df (pandas.DataFrame): the injury dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the InjuryVectorizer
        """
        narrative_vocab = Vocabulary(add_unk=True)
        injury_level_vocab = Vocabulary(add_unk=False)
        # Add ratings
        for injury_level in sorted(set(injury_df.injury_level)):
            injury_level_vocab.add_token(injury_level)
        # Add top words if count > provided count
        word_counts = Counter()
        for injury in injury_df.NARRATIVE:
            for word in injury.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                narrative_vocab.add_token(word)
        return cls(narrative_vocab, injury_level_vocab)
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a InjuryVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the InjuryVectorizer class
        """
        narrative_vocab = Vocabulary.from_serializable(contents['narrative_vocab'])
        injury_level_vocab = Vocabulary.from_serializable(contents['injury_level_vocab'])
        return cls(narrative_vocab=narrative_vocab, injury_level_vocab=injury_level_vocab)
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'narrative_vocab': self.narrative_vocab.to_serializable(),
                'injury_level_vocab': self.injury_level_vocab.to_serializable()}

###### DataLoader - Grouping the vectorized data points

In [ ]:
from torch.utils.data import DataLoader
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
    ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, 
                            shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

 Feed Forward Neural Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class InjuryClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(InjuryClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

In [ ]:
from argparse import Namespace
args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='model.pth',
    injuries_csv='binary_injury_data.csv',
    save_dir="model_storage/binary_classification_FNN",
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,
    # Runtime options
    cuda=True,
    device='cuda',
)

In [ ]:
import torch
import torch.optim as optim
import pandas as pd

def make_train_state(args):
    return {'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': 1,
            'test_acc': 1}

train_state = make_train_state(args)
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

# dataset and vectorizer
dataset = InjuryDataset.load_dataset_and_make_vectorizer(args.injuries_csv)
vectorizer = dataset.get_vectorizer()
# model
classifier = InjuryClassifier(num_features=len(vectorizer.narrative_vocab))
classifier = classifier.to(args.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

In [ ]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [ ]:
import numpy as np
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

In [ ]:
dataset.set_split('test')
batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()
for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index + 1)
    # compute the accuracy
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index + 1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.528
Test Accuracy: 74.22


### The model achieved 74% accuracy in prediction.

In [ ]:
def predict_rating(inj_narrative, classifier, vectorizer,decision_threshold=0.5):
    """Predict the rating of a review
    Args:
        inj_narrative (str): the text of the injury narrative
        classifier (InjuryClassifier): the trained model
        vectorizer (InjuryVectorizer): the corresponding vectorizer
        decision_threshold (float): The numerical boundary which
                                    separates the rating classes
    """
    inj_narrative = preprocess_text(inj_narrative)
    vectorized_narrative = torch.tensor(vectorizer.vectorize(inj_narrative)).to(args.device)
    result = classifier(vectorized_narrative.view(1, -1))
    probability_value = torch.sigmoid(result).item()
    index = 1
    if probability_value < decision_threshold:
        index = 0
    return vectorizer.injury_level_vocab.lookup_index(index)

In [ ]:
def preprocess_text(text, flg_lemm = True,lst_stopwords = None):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?/0-9]+", r" ", text)
    
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    
    #lemmatize
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
    
    #remove stopwords
    if lst_stopwords is not None:
        lst_text= [word for word in lst_text if word not in 
                    lst_stopwords]
        
    ## back to string from list
    text = " ".join(lst_text)   
    
    return text

### Example of an correct prediction

In [ ]:
test_narrative = "The EE was in the processof removing a tapered bushing from the seat in the base of #4 Mill. As he was tightening the bolts into the jacking holes of the bushing with an air impact wrench bushing broke causing the wrench to become vert- ically propelled. causing EE R hand to be crushed in between wrench handle & bottom of mill table."
prediction = predict_rating(test_narrative, classifier, vectorizer)
print("{} > {}".format(test_narrative, prediction))

The EE was in the processof removing a tapered bushing from the seat in the base of #4 Mill. As he was tightening the bolts into the jacking holes of the bushing with an air impact wrench bushing broke causing the wrench to become vert- ically propelled. causing EE R hand to be crushed in between wrench handle & bottom of mill table. > Major


### Example of an incorrect prediction

In [ ]:
test_narrative = "Employee had been bolting sections of duct work together for the previous couple of days.  The work required awkward postures and excessive force in awkward postures.  Employee has a history of back problems.  On this morning he was unable to get out of his truck upon arriving at the job site"
prediction = predict_rating(test_narrative, classifier, vectorizer)
print("{} > {}".format(test_narrative, prediction))

Employee had been bolting sections of duct work together for the previous couple of days.  The work required awkward postures and excessive force in awkward postures.  Employee has a history of back problems.  On this morning he was unable to get out of his truck upon arriving at the job site > Major
